In [2]:
!pip install pycryptodome

In [3]:
import json
import time
import random
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import base64

def generate_sensor_data():
    return {
        "temperature": round(random.uniform(20, 40), 2),
        "humidity": round(random.uniform(30, 80), 2),
        "timestamp": int(time.time())
    }

def encrypt_data(key, plaintext_bytes):
    cipher = AES.new(key, AES.MODE_GCM)
    ciphertext, tag = cipher.encrypt_and_digest(plaintext_bytes)

    return {
        "nonce": base64.b64encode(cipher.nonce).decode(),
        "ciphertext": base64.b64encode(ciphertext).decode(),
        "tag": base64.b64encode(tag).decode()
    }

def decrypt_data(key, encrypted_packet):
    nonce = base64.b64decode(encrypted_packet["nonce"])
    ciphertext = base64.b64decode(encrypted_packet["ciphertext"])
    tag = base64.b64decode(encrypted_packet["tag"])

    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    plaintext = cipher.decrypt_and_verify(ciphertext, tag)
    return plaintext

# ---------------- MAIN ----------------

AES_KEY = get_random_bytes(32)

sensor_data = generate_sensor_data()
plaintext = json.dumps(sensor_data).encode()

encrypted_packet = encrypt_data(AES_KEY, plaintext)
decrypted = decrypt_data(AES_KEY, encrypted_packet)

print("\nOriginal Sensor Data:")
print(json.dumps(sensor_data, indent=4))

print("\nEncrypted Packet:")
print(json.dumps(encrypted_packet, indent=4))

print("\nDecrypted Sensor Data:")
print(json.dumps(json.loads(decrypted.decode()), indent=4))



Original Sensor Data:
{
    "temperature": 32.74,
    "humidity": 66.59,
    "timestamp": 1764333683
}

Encrypted Packet:
{
    "nonce": "SYfPsVHOYeNy+xXaytDldg==",
    "ciphertext": "E20UWd+UEQoQGXxN9FbLMERvJi4Y3IdEqQJK8jD1wNx2iVE/KgolOq/0bn4dW2p8lRIs+Xr7se6viImHSM5mEH6F",
    "tag": "7m2bPxw23HndSnsZyh6b2g=="
}

Decrypted Sensor Data:
{
    "temperature": 32.74,
    "humidity": 66.59,
    "timestamp": 1764333683
}
